# testing scrolling

In [63]:
# loading the packages

import requests
import pandas as pd
import time

# !pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

# using for selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions

[nltk_data] Downloading package stopwords to /Users/daaa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/daaa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/daaa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daaa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.carousell.sg/categories/sports-equipment-10/bicycles-parts-1900/bicycles-247?addRecent=false&canChangeKeyword=false&includeSuggestions=false&sc=0a0208301a0408bbe172220c0a0862696379636c657378012a110a095f64656c69766572793a02080078012a160a0b636f6c6c656374696f6e7312050a0332343778013204080378013a02180742060801100118004a08200128014001480150005a020801&search=bicycles&searchId=6Oyxkt&sort_by=3&tab=marketplace")


productList = []
time.sleep(2)
i = 0

scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

MAX_TRIES = 10  # Set a maximum number of scrolling attempts in case of infinite scrolling pages.

tries = 0
previous_scroll_height = 0

MAX_SCROLLS = 10  # Change this value to the number of scrolls you want.

scroll_count = 0

while scroll_count < MAX_SCROLLS:
    # Scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    time.sleep(scroll_pause_time)
    
    # Try to find the "Read More" button
    try:
        read_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.D_Mw > div > button")))
        if read_more_button:
            print("Read more button clicked for the", i, "time")
            read_more_button.click()
            time.sleep(3)  # Wait after clicking the button for the content to load
    except:
        # If not found, then continue scrolling
        pass

    # Update the counter
    scroll_count += 1
    i += 1

for x in range(1, 3):
    products = driver.find_elements(By.CSS_SELECTOR, f"#main > div.D_Mf > div > section.D_Mr > div.D_Mw > div > div > div:nth-child({x}) > div")
    for product in products: 
        try:
            listing_link = product.find_element(By.CSS_SELECTOR, "div > div.D_zX.M_tF > a:nth-child(2)").get_attribute('href')
        except:
            try:
                listing_link = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a:nth-child(2)").get_attribute('href')
            except:
                listing_link = None


        try:
            product_name = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a:nth-child(2) > p.D_pw.D_ov.D_px.D_pA.D_pE.D_pH.D_pJ.D_pF.D_pN").text
        except:
            try:
                product_name = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a.D__c.D_pW > div.D__g > div > p").text
            except:
                product_name = None

        try:
            img_url = product.find_element(By.TAG_NAME, "img").get_attribute('src')
        except:
            img_url = None
            
        try:
            listing_upload_date = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a.D__c.D_pW > div.D__g > div > p").text
        except:
            listing_upload_date = None

        data = {
            "listing_link": listing_link,
            "product_name": product_name,
            "img_url": img_url,
            "listing_upload_date": listing_upload_date
        }
        print(data)

        productList.append(data)

driver.close()
driver.quit()

In [ ]:
# everything with None as the product name and listing link is a service ad
# we can remove them from our dataframe later
# lets loop through and collect more data

df = pd.DataFrame(list(productList))
df.head()

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df.to_csv("carousell.csv")

In [ ]:
# scraping product condition and description

df = pd.read_csv("carousell.csv")
df.head()

In [ ]:
def initialize_driver(url):
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    return driver

In [ ]:

def scroll_page(driver, max_scrolls=10):
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1
    scroll_count = 0
    while scroll_count < max_scrolls:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        time.sleep(1)
        
        try:
            read_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.D_Mw > div > button")))
            if read_more_button:
                print("Read more button clicked for the", i, "time")
                read_more_button.click()
                time.sleep(3)
        except:
            pass
        
        scroll_count += 1
        i += 1

In [ ]:
def extract_condition_and_description(link):
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    driver.get(link)

    try:
        condition = driver.find_element(By.XPATH, "//*[@id='FieldSetField-Container-field_condition_value']/div/div/div/div/span").text
    except:
        condition = None

    try:
        driver.find_element(By.XPATH, "//*[@id='FieldSetField-Container-field_listing_details_bp']/div/div[5]/button").click()
        time.sleep(3)
        description = driver.find_element(By.CSS_SELECTOR, "#FieldSetField-Container-field_details_bottom_sheet > div > div.D_aCt").text
    except:
        description = None

    driver.close()
    driver.quit()

    return condition, description

In [ ]:
def close_driver(driver):
    driver.close()
    driver.quit()

In [ ]:
def extract_product_data(driver):
    productList = []

    for x in range(1, 3):
        products = driver.find_elements(By.CSS_SELECTOR, f"#main > div.D_Mf > div > section.D_Mr > div.D_Mw > div > div > div:nth-child({x}) > div")
        
        for product in products: 
            # Extract listing link
            try:
                listing_link = product.find_element(By.CSS_SELECTOR, "div > div.D_zX.M_tF > a:nth-child(2)").get_attribute('href')
            except:
                try:
                    listing_link = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a:nth-child(2)").get_attribute('href')
                except:
                    listing_link = None

            # Extract product name
            try:
                product_name = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a:nth-child(2) > p.D_pw.D_ov.D_px.D_pA.D_pE.D_pH.D_pJ.D_pF.D_pN").text
            except:
                try:
                    product_name = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a.D__c.D_pW > div.D__g > div > p").text
                except:
                    product_name = None

            # Extract image URL
            try:
                img_url = product.find_element(By.TAG_NAME, "img").get_attribute('src')
            except:
                img_url = None

            # Extract listing upload date
            try:
                listing_upload_date = product.find_element(By.CSS_SELECTOR, "div > div.D_zX > a.D__c.D_pW > div.D__g > div > p").text
            except:
                listing_upload_date = None

            data = {
                "listing_link": listing_link,
                "product_name": product_name,
                "img_url": img_url,
                "listing_upload_date": listing_upload_date
            }

            productList.append(data)

    return productList


In [ ]:
# Main execution
# url = "https://www.carousell.sg/categories/sports-equipment-10/bicycles-parts-1900/bicycles-247?addRecent=false&canChangeKeyword=false&includeSuggestions=false&sc=0a0208301a0408bbe172220c0a0862696379636c657378012a110a095f64656c69766572793a02080078012a160a0b636f6c6c656374696f6e7312050a0332343778013204080378013a02180742060801100118004a08200128014001480150005a020801&search=bicycles&searchId=6Oyxkt&sort_by=3&tab=marketplace"
# driver = initialize_driver(url)
# scroll_page(driver, max_scrolls=10)
# productList = extract_product_data(driver)
# close_driver(driver)


############################################################################################################


CHUNK_SIZE = 10

df = pd.read_csv("carousell.csv")

if 'condition' not in df.columns:
    df['condition'] = None
if 'description' not in df.columns:
    df['description'] = None

for i in range(0, len(df), CHUNK_SIZE):
    chunk = df.iloc[i:i+CHUNK_SIZE]
    
    for idx, row in chunk.iterrows():
        try:
            condition, description = extract_condition_and_description(row['listing_link'])
            df.at[idx, 'condition'] = condition
            df.at[idx, 'description'] = description
        except Exception as e:
            print(f"Error processing link at index {idx}: {e}")

    df.to_csv('carousell_updated.csv', index=False)
    print(f"Processed and saved up to index {idx}")

df.to_csv('carousell_updated.csv', index=False)

In [ ]:
df1 = pd.read_csv("carousell_raw.csv")
df1.shape

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1.rename(columns={"Unnamed: 0": "index", "listing_link": "market_link"}, inplace=True)

In [ ]:
df1['market'] = "carousell"

In [ ]:
df1.head()

In [ ]:
from datetime import datetime, timedelta
import re

def convert_relative_time_to_datetime(relative_time_str):
    now = datetime.now()
    match = re.match(r"(\d+) (\w+)", relative_time_str)
    if not match:
        return now

    num, unit = match.groups()
    num = int(num)

    if 'minute' in unit:
        return now - timedelta(minutes=num)
    elif 'hour' in unit:
        return now - timedelta(hours=num)
    elif 'day' in unit:
        return now - timedelta(days=num)
    elif 'week' in unit:
        return now - timedelta(weeks=num)
    elif 'month' in unit:
        return now - timedelta(days=30*num)
    elif 'year' in unit:
        return now - timedelta(days=365*num)
    else:
        return now

df1['listing_upload_date'] = df1['listing_upload_date'].apply(convert_relative_time_to_datetime)

In [ ]:
df1.head()

In [ ]:
df1.to_csv("carousell_cleaned_date.csv", index=False)

In [73]:
df2 = pd.read_csv("carousell_cleaned_date.csv")
df2.head()

,index,market_link,product_name,img_url,listing_upload_date,condition,description,market
0,0,https://www.carousell.sg/p/%F0%9F%94%A5instock...,🔥INSTOCKS🔥 26 inch single speed with Black Rim...,https://media.karousell.com/media/photos/produ...,2023-10-13 11:17:09.026519,Brand new,Description\nVISIT US AT OUR OFFICIAL WEBPAGE:...,carousell
1,1,https://www.carousell.sg/p/%F0%9F%94%A5instock...,🔥INSTOCKS🔥 26 inch 21 Speeds Mountain Bike MTB...,https://media.karousell.com/media/photos/produ...,2023-10-13 11:17:09.026625,Brand new,Description\nVISIT US AT OUR OFFICIAL WEBPAGE:...,carousell
2,8,https://www.carousell.sg/p/fixie-bicycle-12597...,Fixie bicycle,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026628,Like new,Description\nBought for my son but not to his ...,carousell
3,9,https://www.carousell.sg/p/foldable-bike-foldi...,Foldable bike folding bicycle Dahon rifle fnho...,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026631,Brand new,Description\nReady stocks \n\nPrice starts fro...,carousell
4,10,https://www.carousell.sg/p/fnhon-foldable-bicy...,Fnhon foldable bicycle,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026632,Like new,Description\nSelling 2 Fnhon foldable bicycle ...,carousell


In [74]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN  # Default to noun

In [75]:
def preprocess_text(text):
    text = text.lower().replace("description", "").replace(" u ", " you ")

    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = contractions.fix(text)

    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    pos_tags = nltk.pos_tag(words)


    words = [word for word in words if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]

    return ' '.join(words).replace(" u ", " you ")


df2['description'] = df2['description'].apply(preprocess_text)
df2.head()

,index,market_link,product_name,img_url,listing_upload_date,condition,description,market
0,0,https://www.carousell.sg/p/%F0%9F%94%A5instock...,🔥INSTOCKS🔥 26 inch single speed with Black Rim...,https://media.karousell.com/media/photos/produ...,2023-10-13 11:17:09.026519,Brand new,visit you at our official webpage brand new in...,carousell
1,1,https://www.carousell.sg/p/%F0%9F%94%A5instock...,🔥INSTOCKS🔥 26 inch 21 Speeds Mountain Bike MTB...,https://media.karousell.com/media/photos/produ...,2023-10-13 11:17:09.026625,Brand new,visit you at our official webpage mountain bik...,carousell
2,8,https://www.carousell.sg/p/fixie-bicycle-12597...,Fixie bicycle,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026628,Like new,bought for my son but not to his like slight s...,carousell
3,9,https://www.carousell.sg/p/foldable-bike-foldi...,Foldable bike folding bicycle Dahon rifle fnho...,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026631,Brand new,ready stock price start from brand dahon hito ...,carousell
4,10,https://www.carousell.sg/p/fnhon-foldable-bicy...,Fnhon foldable bicycle,https://media.karousell.com/media/photos/produ...,2023-10-13 10:11:09.026632,Like new,sell fnhon foldable bicycle be on the road les...,carousell


In [76]:
df2['description'].head()

0    visit you at our official webpage brand new in...
1    visit you at our official webpage mountain bik...
2    bought for my son but not to his like slight s...
3    ready stock price start from brand dahon hito ...
4    sell fnhon foldable bicycle be on the road les...
Name: description, dtype: object

In [77]:
df2.to_csv("carousell_cleaned_final.csv", index=False)